In [1]:
import pyvista
import lfricflux
import numpy
import mint
import vtk

In [2]:
inputFile = '../data/lfric_atm_thai_hab1_C48_MG_dt-1200p0_intel_64-bit_fast-debug__lfric_diag.nc'
#inputFile = '../data/gungho/original/lfric_diag.nc'
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read form the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [3]:
# create an empty vector field object
x, y = lf.getEdgeLonLat()
vf = lfricflux.VtkVectors(x, y, vector_field_name='flow', cartesian=True)

In [4]:
# read the velocity data
u = mint.NcFieldRead(fileName=inputFile, varName='u_in_w2h').data()
v = mint.NcFieldRead(fileName=inputFile, varName='v_in_w2h').data()

In [12]:
# last dimension is number of edges
print(numpy.fabs(u).sum())
print(numpy.fabs(v).sum())

1339.6191318725191
596.7804462711223


In [6]:
# compute rho on edges 
e2f = mint.NcFieldRead(fileName=inputFile, varName='Mesh2d_edge_face_links').data()
rho_face = mint.NcFieldRead(fileName=inputFile, varName='rho').data()
i0 = e2f[:, 0]
i1 = e2f[:, 1]
nfaces = rho_face.shape[-1]
nedges = e2f.shape[0]
rho_edge = numpy.zeros(list(rho_face.shape[:-1]) + [nedges,], numpy.float32)
rho_edge += 0.5*(rho_face[..., e2f[:, 0]] + rho_face[..., e2f[:, 1]])
rho_u = rho_edge * u
rho_v = rho_edge * v

In [15]:
# set the vector field
vf.setField(u, v)
# set time = 0, elev = 0
vf.setSlice((0, 2))

In [16]:
rho_u_max = numpy.fabs(rho_u).max()

In [17]:
# vector field
arrow = vtk.vtkArrowSource()
glyphs = vtk.vtkGlyph3D()
glyphs.SetScaleModeToScaleByVector()
glyphs.SetColorModeToColorByVector()
glyphs.SetScaleFactor(1./rho_u_max) # max value m/s -> 1 radius
glyphs.SetRange(0., rho_u_max)
glyphs.SetInputData(vf.getMesh())
glyphs.SetSourceConnection(arrow.GetOutputPort())

In [18]:
pyvista.global_theme.cmap = 'jet'
p = pyvista.Plotter(window_size=(900, 800))
p.set_background((0.9, 0.9, 0.9))
p.add_mesh(glyphs)
p.show()

Widget(value="<iframe src='http://localhost:57923/index.html?ui=P_0x187c611e0_1&reconnect=auto' style='width: …

In [11]:
from ipywidgets import interact
import ipywidgets as widgets

def callbackElev(k):
    # copy the new values
    vf.setSlice((0, k))
    print(f'k = {k}')
    vf.vectorField.Modified()

interact(callbackElev, k=widgets.IntSlider(min=0, max=u.shape[1] - 1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='k', max=37), Output()), _dom_classes=('widget-interact',…

<function __main__.callbackElev(k)>